<a href="https://kritikseth.github.io/ipynbtagredirect" target="_parent"><img src="https://raw.githack.com/kritikseth/kritikseth/master/assets/icons/kritik_ipynbtagredirect.svg" alt="Kritik Seth"/></a>

## Importing Packages

In [1]:
# !pip install plotly --upgrade
import math 
import pandas_datareader as web
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM

import matplotlib.pyplot as plt
import plotly.express as px
plt.style.use('fivethirtyeight')

## Getting The Data

### Getting Previous Data

In [2]:
df = web.DataReader('TTM', data_source = 'yahoo', start = '2014-01-01', end = '2020-12-12')
df.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2014-01-02,30.110001,29.370001,30.100000,29.570000,1257600.0,29.431362
2014-01-03,30.000000,29.420000,29.650000,29.850000,1330600.0,29.710051
2014-01-06,29.799999,29.450001,29.790001,29.459999,699300.0,29.321877
2014-01-07,29.750000,29.469999,29.700001,29.530001,808400.0,29.391552
2014-01-08,30.270000,29.750000,29.930000,30.100000,1148200.0,29.958878


In [3]:
df_test = web.DataReader('TTM', data_source = 'yahoo', start = '2020-09-01', end = '2021-02-25')
df_test.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2020-09-01,9.87,9.67,9.74,9.77,1526300,9.77
2020-09-02,10.30,9.91,10.22,10.10,2307300,10.10
2020-09-03,10.24,9.77,10.23,9.98,3736600,9.98
2020-09-04,10.15,9.78,10.00,10.10,1741200,10.10
2020-09-08,9.78,9.36,9.47,9.63,2336200,9.63


## Plotting The Data

In [4]:
plt.figure(figsize=(16, 8))
fig = px.line(df, x = df.index, y = df['Close'])
fig.show()

<Figure size 1152x576 with 0 Axes>

## Making Prediction Model

### Keeping The Necessary Values

In [5]:
train = df.filter(['Close'])
train_data = train.values

test = df_test.filter(['Close'])
test_data = test.values

### Sacling The Values

In [6]:
scaler = MinMaxScaler(feature_range = (0, 1))
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

In [7]:
x_train, x_test, y_train, y_test = [], [], [], []

for i in range(60, len(train_data)):
  x_train.append(train_data[i - 60 : i, 0])
  y_train.append(train_data[i, 0])

for i in range(60, len(test_data)):
  x_test.append(test_data[i - 60 : i, 0])
  y_test.append(test_data[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_test, y_test = np.array(x_test), np.array(y_test)

In [8]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

print('Train Shape:', x_train.shape, y_train.shape)
print('Test Shape:', x_test.shape, y_test.shape)

Train Shape: (1690, 60, 1) (1690,)
Test Shape: (62, 60, 1) (62,)


### Building The LSTM Model

In [9]:
model = Sequential()

model.add(LSTM(50, return_sequences = True, input_shape = (x_train.shape[1], 1)))
model.add(LSTM(50, return_sequences = False))
model.add(Dense(25))
model.add(Dense(1))
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [10]:
model.fit(x_train, y_train, batch_size = 1, epochs = 10)

Epoch 1/10
1690/1690 [==============================] - 13s 5ms/step - loss: 0.0022
Epoch 2/10
1690/1690 [==============================] - 9s 5ms/step - loss: 9.4051e-04
Epoch 3/10
1690/1690 [==============================] - 9s 5ms/step - loss: 5.1124e-04
Epoch 4/10
1690/1690 [==============================] - 9s 5ms/step - loss: 5.0988e-04
Epoch 5/10
1690/1690 [==============================] - 9s 5ms/step - loss: 3.8910e-04
Epoch 6/10
1690/1690 [==============================] - 9s 5ms/step - loss: 3.7619e-04
Epoch 7/10
1690/1690 [==============================] - 9s 5ms/step - loss: 4.0714e-04
Epoch 8/10
1690/1690 [==============================] - 9s 5ms/step - loss: 3.2313e-04
Epoch 9/10
1690/1690 [==============================] - 9s 5ms/step - loss: 3.2607e-04
Epoch 10/10
1690/1690 [==============================] - 9s 5ms/step - loss: 3.4559e-04


### Checking on Test Data

In [11]:
prediction = model.predict(x_test)

# Getting The Values Back in Original Scale
prediction = scaler.inverse_transform(prediction)

In [12]:
test = test[-62:]
test['Predictions'] = prediction
test['Error Precentage'] = ((test['Close'] - test['Predictions']) / test['Close']) * 100
test.head()

,Close,Predictions,Error Precentage
Date,,,
2020-11-25,11.78,11.792220,-0.103738
2020-11-27,12.38,11.885909,3.991042
2020-11-30,11.94,12.470892,-4.446335
2020-12-01,12.57,12.036438,4.244723
2020-12-02,12.64,12.615264,0.195699


In [13]:
plt.figure(figsize=(16, 8))
fig = px.line(test, x = test.index, y = ['Close', 'Predictions'])
fig.show()

<Figure size 1152x576 with 0 Axes>